In [84]:
import os
from ultralytics import YOLO
import numpy as np
import cv2
import pandas as pd
from torch import cuda

In [85]:
model_path = "../models/yolov8m-seg.pt"

In [86]:
# ALL_NAMES:
YOLO(model_path).names

{0: 'person',
 1: 'bicycle',
 2: 'car',
 3: 'motorcycle',
 4: 'airplane',
 5: 'bus',
 6: 'train',
 7: 'truck',
 8: 'boat',
 9: 'traffic light',
 10: 'fire hydrant',
 11: 'stop sign',
 12: 'parking meter',
 13: 'bench',
 14: 'bird',
 15: 'cat',
 16: 'dog',
 17: 'horse',
 18: 'sheep',
 19: 'cow',
 20: 'elephant',
 21: 'bear',
 22: 'zebra',
 23: 'giraffe',
 24: 'backpack',
 25: 'umbrella',
 26: 'handbag',
 27: 'tie',
 28: 'suitcase',
 29: 'frisbee',
 30: 'skis',
 31: 'snowboard',
 32: 'sports ball',
 33: 'kite',
 34: 'baseball bat',
 35: 'baseball glove',
 36: 'skateboard',
 37: 'surfboard',
 38: 'tennis racket',
 39: 'bottle',
 40: 'wine glass',
 41: 'cup',
 42: 'fork',
 43: 'knife',
 44: 'spoon',
 45: 'bowl',
 46: 'banana',
 47: 'apple',
 48: 'sandwich',
 49: 'orange',
 50: 'broccoli',
 51: 'carrot',
 52: 'hot dog',
 53: 'pizza',
 54: 'donut',
 55: 'cake',
 56: 'chair',
 57: 'couch',
 58: 'potted plant',
 59: 'bed',
 60: 'dining table',
 61: 'toilet',
 62: 'tv',
 63: 'laptop',
 64: 'mou

In [87]:
classes_ids = [
    2, # Car
    7, # Truck
    5, # Bus
]

In [93]:
device = "gpu" if cuda.is_available() else "cpu"


In [94]:

class YOLOSegmentation:
    def __init__(self, model_path, device):
        self.model = YOLO(model_path, )
        self.device = device

    def detect(self, img):
        height, width, channels = img.shape

        results = self.model.predict(source=img.copy(), save=False, save_txt=False)
        result = results[0]
        segmentation_contours_idx = []
        for seg in result.masks.xyn:
            seg[:, 0] *= width
            seg[:, 1] *= height
            segment = np.array(seg, dtype=np.int32)
            segmentation_contours_idx.append(segment)
        bboxes = np.array(result.boxes.xyxy.to(self.device), dtype="int")
        class_ids = np.array(result.boxes.cls.to(self.device), dtype="int")
        scores = np.array(result.boxes.conf.to(self.device), dtype="float").round(2)

        # bboxes = np.array(result.boxes.xyxy.cpu(), dtype="int")
        # class_ids = np.array(result.boxes.cls.cpu(), dtype="int")
        # scores = np.array(result.boxes.conf.cpu(), dtype="float").round(2)

        return bboxes, class_ids, segmentation_contours_idx, scores


In [102]:
def extract_contours(folder: str, output_dir: str, Segmentator: YOLOSegmentation, transform_to_drawings=False) -> None:
    for image_name in os.listdir(folder)[:10]:
        if not (image_name.endswith(".jpg") or image_name.endswith(".jpeg")):
            continue
        start_path = folder + '/' + image_name
        end_path = f"{output_dir}/"
        os.makedirs(end_path, exist_ok=True)

        img = cv2.imread(start_path)
        if img is None:
            print(Exception(f"Can't read an image: {image_name}"))
            continue
        img = cv2.resize(img, None, fx=0.5, fy=0.5)

        bboxes, classes, segmentations, scores = Segmentator.detect(img)


        # for bbox, class_id, seg, score in zip(bboxes, classes, segmentations, scores):
        #     (x, y, x2, y2) = bbox
        #
        #     cv2.rectangle(img, (x, y), (x2, y2), (255, 0, 0), 2)
        #     cv2.polylines(img, [seg], True, (0, 0, 255), 4)
        #     edges = cv2.putText(img, str(class_id), (x, y - 10), cv2.FONT_HERSHEY_PLAIN, 2, (0, 0, 255), 2)
        #
        #     image_name_pieces = image_name.split(".")
        #     image_name = image_name_pieces[0] + "_processed" + "." + image_name_pieces[-1]
        #     cv2.imwrite(end_path + "/" + image_name, edges)
        # continue


        for item, object_class in enumerate(classes):
            if object_class not in classes_ids:
                continue
            else:
                break


        points = np.array(segmentations[item])
        bbox = bboxes[item]

        mask = np.zeros(img.shape[:2], dtype=np.uint8)

        cv2.drawContours(mask, [points], -1, (255, 255, 255), -1, cv2.LINE_AA)

        (x, y, x2, y2) = bbox
        # cv2.rectangle(img, (x, y), (x2, y2), (255, 0, 0), 2)


        # print(y, y2, x, x2)

        img = cv2.bitwise_and(img, img, mask=mask)

        if transform_to_drawings:

            blurred_image = cv2.GaussianBlur(img.copy(),(5,5),0)

            img = cv2.Canny(blurred_image, 100, 160)


        img = img[y:y2, x:x2]
        cv2.imwrite(end_path + "/" + image_name, img)


In [103]:
WHOLE_DATA_PATH = "../cars_data"
RESULT_PATH = "../extracted_cars"

In [104]:
for dataset in ['train', 'test']:
    folder = WHOLE_DATA_PATH + "/" + dataset
    result_folder = RESULT_PATH + "/" + dataset

    extract_contours(folder=folder,
                     output_dir=result_folder,
                     Segmentator=YOLOSegmentation(model_path, device),
                     transform_to_drawings=False)



0: 640x640 1 car, 1700.3ms
Speed: 0.0ms preprocess, 1700.3ms inference, 0.0ms postprocess per image at shape (1, 3, 640, 640)

0: 640x640 1 car, 1969.0ms
Speed: 15.6ms preprocess, 1969.0ms inference, 7.0ms postprocess per image at shape (1, 3, 640, 640)

0: 640x640 1 car, 1711.0ms
Speed: 7.0ms preprocess, 1711.0ms inference, 6.0ms postprocess per image at shape (1, 3, 640, 640)

0: 640x640 1 car, 1665.0ms
Speed: 6.0ms preprocess, 1665.0ms inference, 7.0ms postprocess per image at shape (1, 3, 640, 640)

0: 640x640 1 car, 1615.0ms
Speed: 7.0ms preprocess, 1615.0ms inference, 7.0ms postprocess per image at shape (1, 3, 640, 640)

0: 640x640 1 car, 1819.0ms
Speed: 7.0ms preprocess, 1819.0ms inference, 7.0ms postprocess per image at shape (1, 3, 640, 640)

0: 640x640 1 car, 1867.0ms
Speed: 7.0ms preprocess, 1867.0ms inference, 5.0ms postprocess per image at shape (1, 3, 640, 640)

0: 640x640 1 car, 1805.0ms
Speed: 6.0ms preprocess, 1805.0ms inference, 7.0ms postprocess per image at shape 